# Fall 2017

In [7]:
import pandas as pd
import os

In [8]:
import csv

In [9]:
os.getcwd()

'/Users/aldana/Dropbox (RCODI)/09 NSF open data/05 Experiments 2017/02 Hack Fall 2017/01 repo Hack Fall 2017/02 Data/2-4 scripts/01 components'

In [10]:
# Here goes the automatic API downloading

all_functions = set()
classes_dict = dict()
with open("./node_modules/d3/API.md") as f: 
    #required_api_list.append(str_api.strip())
    readme_reader = csv.reader(f)
    readme_reader_iter = iter(readme_reader)
    current_class = 'init'
    next(readme_reader_iter) #skipping header
    for i,row in enumerate(readme_reader_iter):
        if row:
            if  row[0].startswith('## ['):
                current_class = row[0].split(']')[0].split('[')[-1].split('(')[-1].split(')')[0].strip()
                classes_dict[row[0].split(']')[0].split('[')[-1].split('(')[-1].split(')')[0].strip()] = dict()
                current_subclass = '_'
                classes_dict[current_class][current_subclass] = set()
            if row[0].startswith('### ['): 
                current_subclass = row[0].split(']')[0].split('[')[-1].split(' ')[0]
                classes_dict[current_class][current_subclass] = set()
            if '[d3.' in row[0]:
                all_functions.add('.'+row[0].split(']')[0].split('[')[-1].split('.')[-1])
                classes_dict[current_class][current_subclass].add('.'+row[0].split(']')[0].split('[')[-1].split('.')[-1])

package_component_list = list([['package','component']])
for package, semantic_class in classes_dict.items():
    for semantic_class, components in semantic_class.items():
        for component in components:
            package_component_list.append([package,component])

if 'results' not in os.listdir('.'):
    os.mkdir('results')

pd.DataFrame(package_component_list).to_csv('./results/packages_components.csv',index=False,header=False)


In [11]:
d3_components_all =[components for val in classes_dict.values() for v in val.values() for components in list(v)]

In [12]:
os.getcwd()

'/Users/aldana/Dropbox (RCODI)/09 NSF open data/05 Experiments 2017/02 Hack Fall 2017/01 repo Hack Fall 2017/02 Data/2-4 scripts/01 components'

In [13]:
os.listdir('../repository/')

['.DS_Store',
 'IH-Project-2017-aclipfel_webapp_phase1',
 'IH-Project-2017-aclipfel_webapp_phase2',
 'IH-Project-2017-aclipfel_webapp_phase3',
 'IH-Project-2017-aclipfel_webapp_phase4',
 'IH-Project-2017-aclipfel_webapp_phase5',
 'IH-Project-2017-Acreed21_webapp_phase1',
 'IH-Project-2017-Acreed21_webapp_phase2',
 'IH-Project-2017-Acreed21_webapp_phase3',
 'IH-Project-2017-Acreed21_webapp_phase4',
 'IH-Project-2017-Acreed21_webapp_phase5',
 'IH-Project-2017-adcorredorm_webapp_phase1',
 'IH-Project-2017-adcorredorm_webapp_phase2',
 'IH-Project-2017-adcorredorm_webapp_phase3',
 'IH-Project-2017-adcorredorm_webapp_phase4',
 'IH-Project-2017-adcorredorm_webapp_phase5',
 'IH-Project-2017-AkaGar_webapp_phase1',
 'IH-Project-2017-AkaGar_webapp_phase2',
 'IH-Project-2017-AkaGar_webapp_phase3',
 'IH-Project-2017-AkaGar_webapp_phase4',
 'IH-Project-2017-AkaGar_webapp_phase5',
 'IH-Project-2017-aldiazve_webapp_phase1',
 'IH-Project-2017-aldiazve_webapp_phase2',
 'IH-Project-2017-aldiazve_webapp_p

In [27]:
df = pd.DataFrame()

participants_code_path = '../repository/'  #'../../2-1 Raw/05 Participants code/'
for repo_folder in [ repo for repo in os.listdir(participants_code_path) if not repo.startswith('.')]:
    for root, folders, files in os.walk(participants_code_path+repo_folder):
        for file_name in files:
            if file_name.endswith(".js"):
                with open(os.path.join( root,  file_name)) as  file_object:
                    file_content = file_object.read()
                    d3_components = set()
                    #for d3 libraries
                    for d3_component in d3_components_all:
                        if d3_component in file_content:
                            class_d3 = [ '_'.join([d3class,subclass,d3_component]) for d3class, val in classes_dict.items() for subclass,components in val.items() if d3_component in components ]
                            if  class_d3:
                                d3_components.add(class_d3[0])
                                #libraries.append(d3_component)
                    num_components = len(list(d3_components))
                    df = df.append(pd.DataFrame({'components':list(d3_components),"username_folder_name":num_components*[repo_folder],"js_loc_in_each_js":num_components*[len(file_content)]}))

In [29]:
df['username'] = df['username_folder_name'].apply(lambda row: row.split('IH-Project-2017-')[-1].split('_webapp_phase')[0])
df['phase'] = df['username_folder_name'].apply(lambda row: row.split('IH-Project-2017-')[-1].split('_')[-1])

In [35]:
df.to_csv('../../2-3 processed data/03 software metrics/component.csv',index=False)